Connected to base (Python 3.11.5)

In [ ]:
Dp_array = df_itineraries['Demand'].tolist()
demand_i = np.zeros(len(L))

NameError: name 'df_itineraries' is not defined

In [ ]:
df_itineraries

NameError: name 'df_itineraries' is not defined

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  4 14:20:54 2024

@author: suzev
"""

# Import necessary libraries
import gurobipy as gp  # Gurobi optimization library
from gurobipy import *  # Import all Gurobi functions
from gurobipy import GRB  # Import Gurobi constants
import numpy as np  # For numerical operations
import math  # For mathematical functions
import copy  # For copying objects
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For plotting graphs
from sklearn.linear_model import LinearRegression

#name model
m = Model('AirlinePlanningAssingment1_Q2')

#import data Maaike
pop_data_path = r'C:\Users\Marjan\OneDrive\Documents\Study\AE44232o Airline planning and optimisation\AE4423Data2\Group_1.xlsx'
demand_data_path = r'C:\Users\Marjan\OneDrive\Documents\Study\AE44232o Airline planning and optimisation\AE4423Data2\DemandGroup1.xlsx'

# #import data Suze
# pop_data_path = "C:\Users\suzev\OneDrive\Documents\Master\AE4423-20 Airline Planning and Optimisation\Assignment 1\Assignment Q2\Group_1.xlsx"
# demand_data_path = r'C:\Users\suzev\OneDrive\Documents\Master\AE4423-20 Airline Planning and Optimisation\Assignment 1\Assignment 1A\DemandGroup1.xlsx'

# Path to the excel file
excel_file = r"C:\Users\Marjan\Documents\Study\AE44232o Airline planning and optimisation\AE4423Data2\Group_1.xlsx"
sheet_name_flights = "Flights"
sheet_name_itineraries = "Itineraries"
sheet_name_recapture = "Recapture"

# Lees het tabblad in een DataFrame
df_flights = pd.read_excel(excel_file, sheet_name=sheet_name_flights)
df_itineraries = pd.read_excel(excel_file, sheet_name=sheet_name_itineraries)
df_recapture = pd.read_excel(excel_file, sheet_name=sheet_name_recapture)

# Print een voorbeeld van de ingeladen data
print(df_flights)
print(df_itineraries)
print(df_recapture)

#%%

# Sets
L = set(df_flights['Flight No.']) # Set of flights
P = set(df_itineraries[['Flight 1', 'Flight 2']].itertuples(index=False, name=None)) # Set of passenger itineraries (paths)
P_p = set(df_recapture[['From Itinerary', 'To Itinerary']].itertuples(index=False, name=None)) # Set of passenger itineraries (paths) with recapture from itinerary p

print("Set of Flights:", L)
print("Set of Passenger Itineraries (Paths):", P)
print("Set of passenger itineraries (paths) with recapture from itinerary p", P_p)

#%%
# Parameters
average_fares = df_itineraries.groupby(['Origin', 'Destination'])['Price [EUR]'].mean()
fare = average_fares.to_dict() # Average fare for itinerary p
Dp = df_itineraries.groupby(['Origin', 'Destination'])['Demand'].sum() # Daily unconstrained demand for itinerary p
CAPi = df_flights['Capacity'] # Capacity on flight (leg) i
b = df_recapture['Recapture Rate'] # Recapture rate of a pax that desires itinerary p and is allocated to r
itinerary_flights = df_itineraries['Itinerary']

# Computing binary variable delta[i, p] for checking if leg i is in path p
L_list = df_flights['Flight No.'].tolist() 
flight1 = df_itineraries['Flight 1']
flight2 = df_itineraries['Flight 2']
delta_matrix = np.zeros((len(P), len(L)))

# Making delta[i, p] matrix
for i in range(len(L)):
    for j in range(len(P)):
        if flight1[j] == L_list[i]:
            delta_matrix[j][i] = 1
        if flight2[j] == L_list[i]:
            delta_matrix[j][i] = 1

#Computing unconstrained demand Q[i]
Dp_array = df_itineraries['Demand'].tolist()
demand_i = np.zeros(len(L))
for i in L:  
        demand_i[i] = sum(Dp_array[p] * delta_matrix[p,i] for p in range(len(P)))  # Summing demand over all paths

Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-21
    Flight No. Origin Destination Departure     Ready  Capacity
0       EA1005   EGLL        LFPG  09:20:00  10:20:00        70
1       EA1041   EGLL        LFPG  15:20:00  16:30:00       150
2       EA1067   EGLL        LFPG  21:15:00  22:25:00       150
3       EA1102   EGLL        LFPG  12:15:00  13:25:00       150
4       EA1133   EGLL        LFPG  14:45:00  15:55:00       150
..         ...    ...         ...       ...       ...       ...
137     EA5122   EGPH        EIDW  11:30:00  12:30:00        70
138     EA5146   EGPH        EIDW  18:55:00  19:55:00        70
139     EA5178   LROP        EDDF  20:55:00  23:20:00        70
140     EA5215   LICJ        EDDF  19:45:00  22:05:00        70
141     EA5231   LICJ        LIRF  13:00:00  14:15:00       150

[142 rows x 6 columns]
     Itinerary Origin Destination Flight 1 Flight 2  Price [EUR]  Demand
0            0   EGLL        LFPG   EA1005  

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
df_itineraries

,Itinerary,Origin,Destination,Flight 1,Flight 2,Price [EUR],Demand
0,0,EGLL,LFPG,EA1005,NaN,54,64
1,1,EGLL,LFPG,EA1041,NaN,63,34
2,2,EGLL,LFPG,EA1067,NaN,44,9
3,3,EGLL,LFPG,EA1102,NaN,49,56
4,4,EGLL,LFPG,EA1133,NaN,70,123
...,...,...,...,...,...,...,...
417,417,EPWA,EDDT,EA4996,EA3644,189,144
418,418,EPWA,EFHK,EA4996,EA3712,277,51
419,419,EGPH,LFPG,EA5056,EA2697,175,77
420,420,EGPH,EHAM,EA5056,EA3030,188,61


In [ ]:
df_fl

NameError: name 'df_fl' is not defined

In [ ]:
df_flights

,Flight No.,Origin,Destination,Departure,Ready,Capacity
0,EA1005,EGLL,LFPG,09:20:00,10:20:00,70
1,EA1041,EGLL,LFPG,15:20:00,16:30:00,150
2,EA1067,EGLL,LFPG,21:15:00,22:25:00,150
3,EA1102,EGLL,LFPG,12:15:00,13:25:00,150
4,EA1133,EGLL,LFPG,14:45:00,15:55:00,150
...,...,...,...,...,...,...
137,EA5122,EGPH,EIDW,11:30:00,12:30:00,70
138,EA5146,EGPH,EIDW,18:55:00,19:55:00,70
139,EA5178,LROP,EDDF,20:55:00,23:20:00,70
140,EA5215,LICJ,EDDF,19:45:00,22:05:00,70


In [ ]:
Dp

Origin  Destination
EDDF    EDDM           315
        EDDT           322
        EFHK            28
        EGLL           300
        EGPH            48
                      ... 
LIRF    EGLL           193
        EHAM            70
        LFPG           118
        LICJ           203
LROP    EDDF            29
Name: Demand, Length: 135, dtype: int64

In [ ]:
# Parameters
average_fares = df_itineraries.groupby(['Origin', 'Destination'])['Price [EUR]'].mean()
fare = average_fares.to_dict() # Average fare for itinerary p
Dp = df_itineraries.groupby(['Origin', 'Destination'])['Demand'].sum() # Daily unconstrained demand for itinerary p
CAPi = df_flights['Capacity'] # Capacity on flight (leg) i
b = df_recapture['Recapture Rate'] # Recapture rate of a pax that desires itinerary p and is allocated to r
itinerary_flights = df_itineraries['Itinerary']

# Computing binary variable delta[i, p] for checking if leg i is in path p
L_list = df_flights['Flight No.'].tolist() 
flight1 = df_itineraries['Flight 1']
flight2 = df_itineraries['Flight 2']
delta_matrix = np.zeros((len(P), len(L)))

# Making delta[i, p] matrix
for i in range(len(L)):
    for j in range(len(P)):
        if flight1[j] == L_list[i]:
            delta_matrix[j][i] = 1
        if flight2[j] == L_list[i]:
            delta_matrix[j][i] = 1


# Vraag per pad (direct uit de DataFrame halen)
Dp = df_itineraries['Demand'].tolist()

# Berekening van de vraag per vlucht
demand_i = np.zeros(len(L))  # Dit gaat de vraag per vlucht bevatten

for i, flight in enumerate(L):  # Itereren over alle vluchten
    demand_i[i] = sum(Dp[p] * delta_matrix[p, i] for p in range(len(P)))  # Som van de vraag gewogen door delta_matrix

# Output de vraag per vlucht
print("Vraag per vlucht (demand_i):")
print(demand_i)
# delta = {}
# for _, row in df_itineraries.iterrows():
#     itinerary = row['Itinerary']
#     if pd.notna(row['Flight 1']):  # If Flight 1 is not NaN, add to delta
#         delta[(row['Flight 1'], itinerary)] = 1
#     if pd.notna(row['Flight 2']):  # If Flight 2 is not NaN, add to delta
#         delta[(row['Flight 2'], itinerary)] = 1  # Flight-path inclusion

# #%%
# # Define δp as a dictionary: (itinerary, flight) -> 1 if flight is part of the itinerary, else 0
# delta = {
#     (itinerary, flight): 1 if flight in flights else 0
#     for itinerary, flights in itinerary_flights.items()  # Iterate over each itinerary and its flights
#     for flight in df_flights['Flight No.'].unique()  # Iterate over all unique flights in df_flights
# }

# print("Delta Mapping (δp):")
# print(δp)

Vraag per vlucht (demand_i):
[ 64.  34.   9.  56. 123.  25.  43.  74.  90. 105.  57.  38.  93. 404.
 561. 401. 911. 155. 110. 116.  70. 117.   9. 120.  75.  66.  16. 118.
  27.  43.   0. 306. 276. 319. 356.  47. 287.  95.   4.   5.  65.  26.
   0.  31. 309. 305. 253. 356. 319. 120.  98. 304. 986. 452. 447. 253.
 803.  43. 378.  91. 480. 104.  87. 170. 291. 278.  82. 465. 249. 338.
  41.  74. 316. 192.  67. 108. 431. 219. 311. 784. 395. 416. 261.  95.
 231. 403.  91. 117. 946.  60. 401. 444. 530. 214. 234. 595. 463.   3.
 237. 399.  17. 200. 354. 202. 287. 162.  44.  51. 856. 203. 167.  32.
 171. 127. 527.  53.  31.  72.  65. 195. 210. 503. 558. 423. 350. 170.
 315.  44.   9.  34. 111. 320. 195. 522.  56. 209.  70.  73.  34.  29.
  36. 203.]


In [ ]:
# Parameters
average_fares = df_itineraries.groupby(['Origin', 'Destination'])['Price [EUR]'].mean()
fare = average_fares.to_dict() # Average fare for itinerary p
Dp = df_itineraries.groupby(['Origin', 'Destination'])['Demand'].sum() # Daily unconstrained demand for itinerary p
CAPi = df_flights['Capacity'] # Capacity on flight (leg) i
b = df_recapture['Recapture Rate'] # Recapture rate of a pax that desires itinerary p and is allocated to r
itinerary_flights = df_itineraries['Itinerary']

# Computing binary variable delta[i, p] for checking if leg i is in path p
L_list = df_flights['Flight No.'].tolist() 
flight1 = df_itineraries['Flight 1']
flight2 = df_itineraries['Flight 2']
delta_matrix = np.zeros((len(P), len(L)))

# Making delta[i, p] matrix
for i in range(len(L)):
    for j in range(len(P)):
        if flight1[j] == L_list[i]:
            delta_matrix[j][i] = 1
        if flight2[j] == L_list[i]:
            delta_matrix[j][i] = 1


# Vraag per pad (direct uit de DataFrame halen)
Dp = df_itineraries['Demand'].tolist()

# Berekening van de vraag per vlucht
demand_i = np.zeros(len(L))  # Dit gaat de vraag per vlucht bevatten

for i, flight in enumerate(L):  # Itereren over alle vluchten
    demand_i[i] = sum(Dp[p] * delta_matrix[p, i] for p in range(len(P)))  # Som van de vraag gewogen door delta_matrix

# Output de vraag per vlucht
print("Vraag per vlucht (demand_i):")
print(demand_i)
# delta = {}
# for _, row in df_itineraries.iterrows():
#     itinerary = row['Itinerary']
#     if pd.notna(row['Flight 1']):  # If Flight 1 is not NaN, add to delta
#         delta[(row['Flight 1'], itinerary)] = 1
#     if pd.notna(row['Flight 2']):  # If Flight 2 is not NaN, add to delta
#         delta[(row['Flight 2'], itinerary)] = 1  # Flight-path inclusion

# #%%
# # Define δp as a dictionary: (itinerary, flight) -> 1 if flight is part of the itinerary, else 0
# delta = {
#     (itinerary, flight): 1 if flight in flights else 0
#     for itinerary, flights in itinerary_flights.items()  # Iterate over each itinerary and its flights
#     for flight in df_flights['Flight No.'].unique()  # Iterate over all unique flights in df_flights
# }

# print("Delta Mapping (δp):")
# print(δp)

Vraag per vlucht (demand_i):
[ 64.  34.   9.  56. 123.  25.  43.  74.  90. 105.  57.  38.  93. 404.
 561. 401. 911. 155. 110. 116.  70. 117.   9. 120.  75.  66.  16. 118.
  27.  43.   0. 306. 276. 319. 356.  47. 287.  95.   4.   5.  65.  26.
   0.  31. 309. 305. 253. 356. 319. 120.  98. 304. 986. 452. 447. 253.
 803.  43. 378.  91. 480. 104.  87. 170. 291. 278.  82. 465. 249. 338.
  41.  74. 316. 192.  67. 108. 431. 219. 311. 784. 395. 416. 261.  95.
 231. 403.  91. 117. 946.  60. 401. 444. 530. 214. 234. 595. 463.   3.
 237. 399.  17. 200. 354. 202. 287. 162.  44.  51. 856. 203. 167.  32.
 171. 127. 527.  53.  31.  72.  65. 195. 210. 503. 558. 423. 350. 170.
 315.  44.   9.  34. 111. 320. 195. 522.  56. 209.  70.  73.  34.  29.
  36. 203.]


In [ ]:
demand

NameError: name 'demand' is not defined

In [ ]:
demand_i

array([ 64.,  34.,   9.,  56., 123.,  25.,  43.,  74.,  90., 105.,  57.,
        38.,  93., 404., 561., 401., 911., 155., 110., 116.,  70., 117.,
         9., 120.,  75.,  66.,  16., 118.,  27.,  43.,   0., 306., 276.,
       319., 356.,  47., 287.,  95.,   4.,   5.,  65.,  26.,   0.,  31.,
       309., 305., 253., 356., 319., 120.,  98., 304., 986., 452., 447.,
       253., 803.,  43., 378.,  91., 480., 104.,  87., 170., 291., 278.,
        82., 465., 249., 338.,  41.,  74., 316., 192.,  67., 108., 431.,
       219., 311., 784., 395., 416., 261.,  95., 231., 403.,  91., 117.,
       946.,  60., 401., 444., 530., 214., 234., 595., 463.,   3., 237.,
       399.,  17., 200., 354., 202., 287., 162.,  44.,  51., 856., 203.,
       167.,  32., 171., 127., 527.,  53.,  31.,  72.,  65., 195., 210.,
       503., 558., 423., 350., 170., 315.,  44.,   9.,  34., 111., 320.,
       195., 522.,  56., 209.,  70.,  73.,  34.,  29.,  36., 203.])

In [ ]:
for i, flight in enumerate(L):  # Itereren over alle vluchten
    demand_i[i] = sum(Dp[p] * delta_matrix[p, i] for p in range(len(P)))  # Som van de vraag gewogen door delta_matrix

In [ ]:
demand

NameError: name 'demand' is not defined

In [ ]:
demand_i

array([ 64.,  34.,   9.,  56., 123.,  25.,  43.,  74.,  90., 105.,  57.,
        38.,  93., 404., 561., 401., 911., 155., 110., 116.,  70., 117.,
         9., 120.,  75.,  66.,  16., 118.,  27.,  43.,   0., 306., 276.,
       319., 356.,  47., 287.,  95.,   4.,   5.,  65.,  26.,   0.,  31.,
       309., 305., 253., 356., 319., 120.,  98., 304., 986., 452., 447.,
       253., 803.,  43., 378.,  91., 480., 104.,  87., 170., 291., 278.,
        82., 465., 249., 338.,  41.,  74., 316., 192.,  67., 108., 431.,
       219., 311., 784., 395., 416., 261.,  95., 231., 403.,  91., 117.,
       946.,  60., 401., 444., 530., 214., 234., 595., 463.,   3., 237.,
       399.,  17., 200., 354., 202., 287., 162.,  44.,  51., 856., 203.,
       167.,  32., 171., 127., 527.,  53.,  31.,  72.,  65., 195., 210.,
       503., 558., 423., 350., 170., 315.,  44.,   9.,  34., 111., 320.,
       195., 522.,  56., 209.,  70.,  73.,  34.,  29.,  36., 203.])

In [ ]:
len(demand_i)

142

In [ ]:
Dp

[64,
 34,
 9,
 56,
 123,
 25,
 43,
 74,
 90,
 105,
 57,
 38,
 93,
 63,
 219,
 13,
 5,
 155,
 110,
 116,
 70,
 117,
 9,
 120,
 75,
 66,
 16,
 118,
 27,
 43,
 0,
 59,
 24,
 3,
 17,
 47,
 56,
 95,
 4,
 5,
 65,
 26,
 0,
 31,
 25,
 29,
 32,
 15,
 59,
 6,
 14,
 12,
 114,
 120,
 54,
 43,
 32,
 43,
 9,
 36,
 44,
 32,
 15,
 48,
 12,
 3,
 24,
 44,
 55,
 50,
 41,
 55,
 45,
 15,
 15,
 54,
 21,
 23,
 46,
 89,
 69,
 65,
 23,
 15,
 32,
 45,
 61,
 33,
 53,
 60,
 28,
 44,
 48,
 29,
 36,
 55,
 50,
 3,
 20,
 28,
 17,
 20,
 44,
 71,
 57,
 55,
 44,
 51,
 89,
 203,
 167,
 32,
 171,
 127,
 69,
 53,
 31,
 72,
 65,
 195,
 24,
 45,
 41,
 33,
 77,
 34,
 25,
 44,
 9,
 34,
 111,
 28,
 195,
 44,
 56,
 48,
 70,
 73,
 34,
 29,
 36,
 203,
 49,
 98,
 47,
 50,
 74,
 23,
 158,
 31,
 153,
 140,
 38,
 66,
 144,
 190,
 11,
 100,
 258,
 92,
 207,
 48,
 8,
 26,
 0,
 6,
 32,
 18,
 73,
 20,
 36,
 20,
 8,
 90,
 4,
 21,
 16,
 6,
 6,
 7,
 40,
 44,
 4,
 14,
 90,
 32,
 34,
 25,
 26,
 8,
 33,
 24,
 37,
 7,
 19,
 127,
 21,
 10,
 32,
 

In [ ]:
for i in L:  # Itereren over alle vluchten
    demand_i[i] = sum(Dp[p] * delta_matrix[p, i] for p in range(len(P)))  # Som van de vraag gewogen door delta_matrix

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
for i in range(len(L)):  # Itereren over alle vluchten
    demand_i[i] = sum(Dp[p] * delta_matrix[p, i] for p in range(len(P)))  # Som van de vraag gewogen door delta_matrix

In [ ]:
deman

NameError: name 'deman' is not defined

In [ ]:
demand_i

array([ 64.,  34.,   9.,  56., 123.,  25.,  43.,  74.,  90., 105.,  57.,
        38.,  93., 404., 561., 401., 911., 155., 110., 116.,  70., 117.,
         9., 120.,  75.,  66.,  16., 118.,  27.,  43.,   0., 306., 276.,
       319., 356.,  47., 287.,  95.,   4.,   5.,  65.,  26.,   0.,  31.,
       309., 305., 253., 356., 319., 120.,  98., 304., 986., 452., 447.,
       253., 803.,  43., 378.,  91., 480., 104.,  87., 170., 291., 278.,
        82., 465., 249., 338.,  41.,  74., 316., 192.,  67., 108., 431.,
       219., 311., 784., 395., 416., 261.,  95., 231., 403.,  91., 117.,
       946.,  60., 401., 444., 530., 214., 234., 595., 463.,   3., 237.,
       399.,  17., 200., 354., 202., 287., 162.,  44.,  51., 856., 203.,
       167.,  32., 171., 127., 527.,  53.,  31.,  72.,  65., 195., 210.,
       503., 558., 423., 350., 170., 315.,  44.,   9.,  34., 111., 320.,
       195., 522.,  56., 209.,  70.,  73.,  34.,  29.,  36., 203.])

In [ ]:
DELTA

NameError: name 'DELTA' is not defined

In [ ]:
delta_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])